In [1]:
# Import libraries
import os
import sys
import logging
import pickle # remove
import shutil

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, "..")) # absolute path for ekr_rag directory
repo_dir = os.path.abspath(os.path.join(kit_dir, "../../../")) # absolute path for starter-kit directory
print(kit_dir)
print(repo_dir)

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.document_retrieval import DocumentRetrieval

CONFIG_PATH = os.path.join(kit_dir,'config.yaml')
PERSIST_DIRECTORY = os.path.join(kit_dir,f"data/my-vector-db")
#save_location = kit_dir + "/data/my-vector-db"

/Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit/workshops/ai_engineer_2024/ekr_rag
/Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit


/Users/petrojm/miniconda3/envs/complex_rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Petro
/Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit


In [2]:
from PyPDF2 import PdfReader
from langchain_community.document_loaders import UnstructuredPDFLoader

# Specify PDF file
pdf_file = kit_dir + '/data/tmp/2405.07518v1_1page.pdf'

# Initialize DocumentRetrieval class
documentRetrieval =  DocumentRetrieval()
print(documentRetrieval.loaders['pdf'])

# Get pdf text
raw_text = []
meta_data = []
if documentRetrieval.loaders['pdf'] == "unstructured":
    loader = UnstructuredPDFLoader(pdf_file)
    docs_unstructured = loader.load()
    for doc in docs_unstructured:
        raw_text.append(doc.page_content)
        meta_data.append({"filename": pdf_file})
elif documentRetrieval.loaders['pdf'] == "pypdf2":
    pdf_reader = PdfReader(pdf_file)
    for page in pdf_reader.pages:
        raw_text.append(page.extract_text())
        meta_data.append({"filename": pdf_file})#, "page": page_number})
else:
    raise ValueError(f"{self.documentRetrieval.loaders['pdf']} is not a valid pdf loader")


#raw_text, meta_data = documentRetrieval.get_data_for_splitting(docs)
#with open(kit_dir+'/streamlit/'+'raw_text.pkl', 'rb') as file:
#    raw_text = pickle.load(file)
#with open(kit_dir+'/streamlit/'+'meta_data.pkl', 'rb') as file:
#    meta_data = pickle.load(file)

# Get the text chunks
text_chunks = documentRetrieval.get_text_chunks_with_metadata(docs=raw_text, meta_data=meta_data)
print(text_chunks[0])
print('nb of chunks: %d'%len(text_chunks))
#print(type(text_chunks[0])) # list of langchain_core.documents.base.Document
#print(text_chunks[0].page_content)

pypdf2
page_content='Source: /Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit/workshops/ai_engineer_2024/ekr_rag/data/tmp/2405, Text: \nSambaNova SN40L: Scaling the AI Memory Wall\nwith Dataﬂow and Composition of Experts\nRaghu Prabhakar, Ram Sivaramakrishnan, Darshan Gandhi, Yun Du, Mingran Wang, Xiangyu Song, Kejie Zhang,\nTianren Gao, Angela Wang, Karen Li, Yongning Sheng, Joshua Brot, Denis Sokolov, Apurv Vivek, Calvin Leung,\nArjun Sabnis, Jiayu Bai, Tuowen Zhao, Mark Gottscho, David Jackson, Mark Luttrell, Manish K. Shah, Edison Chen,\nKaizhao Liang, Swayambhoo Jain, Urmish Thakker, Dawei Huang, Sumti Jairath, Kevin J. Brown, Kunle Olukotun\nSambaNova Systems, Inc.\nﬁrst.last@sambanova.ai\nAbstract —Monolithic large language models (LLMs) like\nGPT-4 have paved the way for modern generative AI\napplications. Training, serving, and maintaining monolithic\nLLMs at scale, however, remains prohibitively expensive\nand challenging. The disproportionat

In [3]:
# Create vector store
embeddings = documentRetrieval.load_embedding_model()
if os.path.exists(PERSIST_DIRECTORY):
    shutil.rmtree(PERSIST_DIRECTORY)
    print(f"The directory Chroma has been deleted.")
#vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=None)
vectorstore = documentRetrieval.create_vector_store(text_chunks, embeddings, output_db=PERSIST_DIRECTORY)

# Create conversation chain
documentRetrieval.init_retriever(vectorstore)
conversation = documentRetrieval.get_qa_retrieval_chain()

/Users/petrojm/miniconda3/envs/complex_rag/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
2024-06-20 23:45:54,290 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-06-20 23:45:55,926 [INFO] - Use pytorch device: cpu


max_seq_length  512
The directory Chroma has been deleted.


2024-06-20 23:45:56,424 [INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-06-20 23:46:02,715 [INFO] - Vector store saved to /Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit/workshops/ai_engineer_2024/ekr_rag/data/my-vector-db


In [4]:
# Ask questions about your data
user_question = "What is composition of experts (CoE)?"

response = conversation.invoke({"question":user_question})
#print(response)
print(response['question'])
print(response['answer'])
print(len(response['source_documents']))
print('\nSource #1:')
print(response['source_documents'][0].page_content)
print(response['source_documents'][0].metadata)

print('\nSource #2:')
print(response['source_documents'][1].page_content)
print(response['source_documents'][1].metadata)

print('\nSource #3:')
print(response['source_documents'][2].page_content)
print(response['source_documents'][2].metadata)


2024-06-20 23:46:06,945 [WARNING] - Number of requested results 15 is greater than number of elements in index 6, updating n_results = 6


What is composition of experts (CoE)?
Composition of Experts (CoE) is a modular approach that involves combining many smaller expert models, each with fewer parameters, to match or exceed the capabilities of monolithic large language models (LLMs).
3

Source #1:
Source: /Users/petrojm/Documents/projects/workshops/ai_engineer_2024/forked/ai-starter-kit/workshops/ai_engineer_2024/ekr_rag/data/tmp/2405, Text: 
research has shown that a composition of many smaller expert
models, each with several orders of magnitude fewer parameters,
can match or exceed the capabilities of monolithic LLMs.
Composition of Experts (CoE) is a modular approach that lowers
the cost and complexity of training and serving. However, this
approach presents two key challenges when using conventional
hardware: (1) without fused operations, smaller models have
lower operational intensity, which makes high utilization more
challenging to achieve; and (2) hosting a large number of models
can be either prohibitively expe